In [4]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
load_dotenv()

end_point = os.getenv("AZURE_OPENAI_GPT_4O_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_GPT_4O_API_KEY")
api_version = os.getenv("AZURE_OPENAI_GPT_4O_API_VERSION")
deployment = os.getenv("AZURE_OPENAI_GPT_4O_DEPLOYMENT_NAME")


llm_client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=end_point,
    api_key= api_key
)

response = llm_client.chat.completions.create(
    messages = [
        {"role":"system","content":"You are a AI assisstant."},
        {"role":"user","content":"Tell me about large learning models"},   
    ],
    max_tokens=256,
    temperature = 0.99,
    model = "gpt-4o",
)

print(response.choices[0].message.content)

Large learning models, also known as Large Language Models (LLMs), are advanced machine learning systems designed to process and generate human-like text. They are typically based on deep learning architectures, particularly transformer architectures, and are trained on massive amounts of textual data. These models have revolutionized natural language processing (NLP) by achieving state-of-the-art performance on a variety of language-based tasks.

### Key Characteristics of Large Learning Models

#### 1. **Scale**:
   - Large learning models are characterized by their size, in terms of both the number of parameters (millions to hundreds of billions) and the scale of the data used for training.
   - The size of the model allows it to capture nuanced patterns and complex relationships in language.

#### 2. **Transformer Architecture**:
   - Most large learning models are built on the transformer architecture, which was introduced in the paper "Attention Is All You Need" (Vaswani et al., 

In [5]:
from sentence_transformers import SentenceTransformer, CrossEncoder
import chromadb

# Load embedding and cross-encoder models
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# Initialize ChromaDB client
client = chromadb.PersistentClient(path="knowledge_base")
collection = client.get_or_create_collection(name="GENAI_collection")

def fetch_top_relevant_queries(query, collection, top_k=10):
    query_embedding = embedding_model.encode(query).tolist()
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)
    return results['documents'][0], results['metadatas'][0], results['distances'][0]

def retrieve_relevant_content(query):
    documents, metadata, scores = fetch_top_relevant_queries(query, collection, top_k=10)
    # Compute cross-encoder scores
    scored_queries = []
    for doc, meta, score in zip(documents, metadata, scores):
        doc_text = doc.replace("\n", " ")
        cross_score = cross_encoder.predict([(query, doc_text)])[0]
        scored_queries.append({"document": doc_text, "metadata": meta, "similarity_score": score, "cross_encoder_score": cross_score})

    # Sort by cross-encoder score and take top 3
    top_docs = sorted(scored_queries, key=lambda x: x['cross_encoder_score'], reverse=True)[:3]

    # Combine top 3 documents into a single paragraph
    combined_text = " ".join([item['document'] for item in top_docs])

    return combined_text


retrieve_relevant_content("What is l and t doing for sustainability of environemtn?")

'green business | l&t sustainability sustainability our approach materiality planet energy & emissions water circular economy biodiversity innovation green business climate strategy people csr - health csr - skill building csr - education jyoti csr - water & sanitation health & safety volunteering prayas trust governance green    business sustainability planet green business l&t offers a bouquet of sustainable solutions which helps its customers go green. these solutions, covered under our green business, are enabled through advanced technology, and centred around clean energy, clean mobility, water and sanitation, green infra and other areas linked to a green future. with our green businesses, we help our customers to lower carbon emissions, improve water use and recycling, enhance energy efficiency, reduce air pollution, and promote resource conservation. overall, we help preserve the environment as well as improve public well-being. climate | l&t sustainability sustainability our ap

In [6]:
def enhance_prompt(query):
    """Enhances the user query to improve retrieval accuracy."""
    chat_completion = llm_client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are an AI assistant for L&T group. Your task is to refine user queries for better information retrieval. Make the query clearer, more specific, and structured to help retrieve relevant documents accurately."},
            {"role": "user", "content": f"Enhance this query for better retrieval: {query}"}
        ],
        model="gpt-4o",
        max_tokens=50
    )
    return chat_completion.choices[0].message.content.strip()

def enhanced_llm_call(user_query):
    """Enhances query, retrieves relevant content, and gets final response."""
    
    # Step 1: Enhance the query for better retrieval
    refined_query = enhance_prompt(user_query)

    # Step 2: Retrieve relevant content using the enhanced query
    relevant_content = retrieve_relevant_content(refined_query)

    # Step 3: Send both the retrieved content and the original user question to LLM
    chat_completion = llm_client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are an AI assistant. Answer the user's question using the given context."},
            {"role": "user", "content": f"Context:\n{relevant_content}\n\nUser Question: {user_query}"}
        ],
        model="gpt-4o",
        max_tokens=512
    )

    return chat_completion.choices[0].message.content

# Example Usage
user_query = "Who is your chairman"
final_output = enhanced_llm_call(user_query)
print(final_output)

As of my knowledge cutoff in October 2023, the Chairman of Larsen & Toubro (L&T) is **A.M. Naik** (Anil Manibhai Naik). He has long been associated with the company and is a prominent industrial leader in India. If you're seeking the most current information, I recommend checking L&T's official website or their latest corporate announcements for updates.
